# LDAP admin
---

In [ ]:
# epena: missing create home/<usr> 

In [23]:
import re, time
import pandas as pd
import ipywidgets as w
from IPython import display as d

from ldap3 import Server, Connection, ALL, MODIFY_REPLACE
from passlib.hash import ldap_salted_sha1

ldap_base = 'dc=pl,dc=eso,dc=org'
user_base = 'ou=users,' + ldap_base
group_base = 'ou=groups,' + ldap_base
path_to_userlist = 'official.csv'

server = Server('ldap://ldap-server')

In [24]:
import getpass
password = getpass.getpass('LDAP admin password:')

password='admin'

LDAP admin password:········


In [25]:

connection = Connection(server, 'cn=admin,{}'.format(ldap_base), password, auto_bind=True)

result_codes = {
    0: 'Success',
    1: 'Username must be 2-8 characters long.',
    2: 'Username already exists.',
    3: 'Incorrect password.',
    4: 'User does not exist.',
    5: 'LDAP error.',
    6: 'Group does not exist.',
    7: 'User already in group.',
}

In [26]:
def get_uid():
    uid_list = []
    connection.search(ldap_base, '(uidNumber=*)', attributes=['uidNumber'])
    for entry in connection.entries:
        uid_list.append(entry.entry_attributes_as_dict['uidNumber'][0])
    return max(uid_list) + 1

In [27]:
def add_user(user_name, first_name, last_name):
    if len(user_name) < 2 or len(user_name) > 8:
        return 1
    connection.search(user_base, '(cn={})'.format(user_name))
    if len(connection.entries) > 0:
        return 2
    user_uid = get_uid()
    dn = 'cn={},{}'.format(user_name, user_base)
    modlist = {
        'cn': user_name,
        'gidNumber': 500,
        'homeDirectory': '/home/users/{}'.format(user_name),
        'objectClass': ['inetOrgPerson', 'posixAccount', 'top'],
        'sn': last_name,
        'givenName': first_name,
        'uid': user_name,
        'uidNumber': user_uid,
        'userPassword': ldap_salted_sha1.hash('Bnice2me'),
    }

    connection.add(dn, attributes=modlist)
    return 0

In [28]:
def delete_user(user_name):
    connection.delete('cn={},{}'.format(user_name, user_base))
    return 0

In [29]:
def change_password(user_name, new_pass, old_pass=None):
    dn = 'cn={},{}'.format(user_name, user_base)
    connection.search(ldap_base, '(cn={})'.format(user_name), attributes=['userPassword'])
    if len(connection.entries) == 0:
        return 4
    
    if old_pass:
        current_pass = connection.entries[0].entry_attributes_as_dict['userPassword'][0]
        if not ldap_salted_sha1.verify(old_pass, current_pass):
            return 3
    
    if connection.modify('cn={},{}'.format(user_name, user_base), {
        'userPassword': [(MODIFY_REPLACE, [ldap_salted_sha1.hash(new_pass)])]
    }):
        return 0
    return 5

In [30]:
def list_from_ldap():
    df = {'Username': [],
          'First Name': [],
          'Last Name': [],
          'UID': []}
    connection.search(user_base, '(cn=*)', attributes=['cn', 'givenName', 'sn', 'uidNumber'])
    for entry in connection.entries:
        df['Username'].append(entry['cn'][0])
        df['First Name'].append(entry['givenName'][0])
        df['Last Name'].append(entry['sn'][0])
        df['UID'].append(entry['uidNumber'][0])
    return pd.DataFrame(df)

In [31]:
def get_users_from_group(group):
    connection.search(group_base, '(cn={})'.format(group), attributes=['memberUid'])
    return connection.entries[0].entry_attributes_as_dict['memberUid']

In [32]:
def add_user_to_group(user_name, group):
    connection.search(ldap_base, '(cn={})'.format(user_name))
    if len(connection.entries) == 0:
        return 4
    
    connection.search(group_base, '(cn={})'.format(group))
    if len(connection.entries) == 0:
        return 6
    
    group_dn = connection.entries[0].entry_dn
    users = get_users_from_group(group)
    if user_name in users:
        return 7
    
    if connection.modify(group_dn, {
        'memberUid': [(MODIFY_REPLACE, users + [user_name])]
    }):
        return 0
    return 5

In [33]:
def is_user_in_group(user, group):
    return user in get_users_from_group(group)

In [34]:
def parse_phonebook(savename, inputname='Phonebook.csv'):
    phonebook = (pd.read_csv(inputname, sep='|')
                 .drop(columns=['Unnamed: 9']))
    phonebook = phonebook[phonebook['ESO Site'] == 'Paranal']
    phonebook = phonebook[phonebook['First Name'].notna()]
    phonebook = phonebook[phonebook['Department'].isin(['MSE'])]
    phonebook = phonebook[phonebook.Email.map(lambda x: len(x)) < 17].reset_index(drop=True)
    phonebook['Username'] = phonebook.Email.map(lambda x: x.split('@')[0])
    phonebook[['Username', 'First Name', 'Name']].to_csv(savename, index=False)

In [35]:
# users = get_users_from_group('qowAdmin')
# users

In [36]:
# connection.search(group_base, '(cn={})'.format('asdf'), attributes=['memberUid'])
# entry = connection.entries[0]
# entry.entry_dn

In [37]:
# connection.modify(entry.entry_dn, {
#     'memberUid': [(MODIFY_REPLACE, users + ['epena'])]
# })

## Add single user:
---

In [38]:
w_username = w.Text(placeholder='username')
w_firstname = w.Text(placeholder='First name')
w_lastname = w.Text(placeholder='Last name')
w_add_user_button = w.Button(description='Add user', icon='plus', button_style='success')
w_add_user_result = w.Label()

def button_add_user_action(*_):
    w_add_user_result.value = result_codes[add_user(
        w_username.value, w_firstname.value, w_lastname.value
    )]

w_add_user_button.on_click(button_add_user_action)
w.VBox([w_username, w_firstname, w_lastname, w_add_user_button, w_add_user_result])

## Add user to group:
---

In [39]:
w_username = w.Text(placeholder='username')
w_group = w.Text(placeholder='group')
w_add_user2g_button = w.Button(description='Add user to group', icon='plus', button_style='success')
w_add_user2g_result = w.Label()

def button_add_user_to_group_action(*_):
    w_add_user2g_result.value = result_codes[add_user_to_group(
        w_username.value, w_group.value
    )]

w_add_user2g_button.on_click(button_add_user_to_group_action)
w.VBox([w_username, w_group, w_add_user2g_button, w_add_user2g_result])

## Change password:
---

In [40]:
w_username = w.Text(placeholder='username')
w_password = w.Password(placeholder='password')
w_mod_pass_button = w.Button(description='Change Password', icon='key', button_style='success')
w_mod_pass_result = w.Label()

def button_mod_pass_action(*_):
    w_mod_pass_result.value = result_codes[change_password(
        w_username.value, w_password.value
    )]

w_mod_pass_button.on_click(button_mod_pass_action)
w.VBox([w.HBox([w_username, w_password]), w_mod_pass_button, w_mod_pass_result])

## Add users:
---

In [41]:
w_filepath = w.Text(value=path_to_userlist)
w_add_bulk_button = w.Button(description='Upload List', icon='arrow-up', button_style='success')
b_add_bulk_result = w.VBox()

# change this function.
def button_add_bulk_action(*_):
    df = pd.read_csv(w_filepath.value)
    r_list = []
    for i, row in df.iterrows():
        r_list.append(add_user(row['Username'], row['First Name'], row['Name']))
    results = []
    for i, tag in enumerate(['Inserted', 'Invalid', 'Already up']):
        results.append(w.Label(value='{} : {}'.format(tag, r_list.count(i))))
    b_add_bulk_result.children = results

w_add_bulk_button.on_click(button_add_bulk_action)
w.VBox([w.HBox([w_filepath, w_add_bulk_button]), b_add_bulk_result])

## List users on the server:
---

In [42]:
w_list_users_button = w.Button(description='List LDAP users', icon='list-ul')

def button_list_users_action(*_):
    with pd.option_context('display.max_rows', None):
        d.display(pd.DataFrame(list_from_ldap()))

w_list_users_button.on_click(button_list_users_action)
w.VBox([w_list_users_button])

,Username,First Name,Last Name,UID
0,espr,Espresso,account,1011
1,jgil,JuanPablo,Gil,1006
2,lroa,Luis,Roa,1025
3,test,Hey,There,1118
4,adiaz,Álvaro,Díaz,1066
5,celao,Cristian,Elao,1070
6,epena,Eduardo,Peña,1004
7,gzins,Gerard,Zins,1113
8,jbaez,Jose,Baez,1016
9,jkolb,Johann,Kolb,1084


## Contrast user lists:
---

In [86]:
w_contrast_button = w.Button(description='Contrast lists')
b_contrast_result = w.VBox()

def button_contrast_action(*_):
    ldap_list = list_from_ldap()
    ldap_list.insert(0, 'from', True)
    csv_list = pd.read_csv(w_filepath.value)
    csv_list.insert(0, 'from', True)
    data = pd.merge(ldap_list, csv_list, how='outer', on='Username', suffixes=('_ldap', '_csv')).fillna(False)
    d.display(d.Markdown('''### Data from LDAP not in CSV:\n\n---'''))
    d.display((data[data['from_ldap'] & ~data['from_csv']])[['Username', 'UID']])
    d.display(d.Markdown('''### Data from CSV not in LDAP:\n\n---'''))
    d.display((data[data['from_csv'] & ~data['from_ldap']])[['Username']])

w_contrast_button.on_click(button_contrast_action)
w.HBox([w_filepath, w_contrast_button])